In [71]:
from itertools import count
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [72]:
sitename = "부산한마음스포츠센터"
name_dic={"구분":[]}
##program 페이지 찾기
html = requests.get("http://hmsports.bisco.or.kr/").text
htmlAll = bs(html,'html.parser')
program_page = htmlAll.find_all("li",{"class":"mn_li1"})[1].find("a")["href"]

In [73]:
##컬럼 지정
my_columns = ["프로그램","반","대상","교육일","시간","정원","교육비","비고"]
dic = {}
for col in my_columns:
    dic[col]=list()

In [74]:
## 각 프로그램의 url 찾기
html = requests.get("http://hmsports.bisco.or.kr/"+program_page).text
htmlAll = bs(html,'html.parser')
pages = htmlAll.find_all("ul",{"class":"depth2"})[-1].find_all("a")
programURLs = [url["href"] for url in pages]
del programURLs[-1]## 통합방과후학교 삭제
del programURLs[4]## 실내골프연습장 삭제
del programURLs[2]## 피트니스실 삭제

In [75]:
for url in programURLs:
    html = requests.get("http://hmsports.bisco.or.kr"+url).text
    htmlAll = bs(html,'html.parser')
    initial_df = pd.read_html(html,header=0)[0]  #표 df로 변환
    current_col = initial_df.columns.tolist()
    for col in current_col:
        for value in initial_df[col]:
            dic[col].append(value)
    for _ in range(len(initial_df[current_col[0]].values)):
        name_dic["구분"].append(htmlAll.find("div",{"id":"print"}).find('h4',{'class':'ctit'}).text)
dic.update(name_dic)
dic

{'프로그램': ['기초종목 육성반',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인수영',
  '장애인체육',
  '장애인 문화교실',
  '장애인 문화교실',
  '장애인 문화교실',
  '장애인 문화교실'],
 '반': ['기초종목 육성반(화,목,토)',
  '수영선수육성반',
  '오전건강수영A(10~1050)',
  '오전건강수영B(10~1050)',
  '오전건강수영C(11~1150)',
  '오전건강수영D(11~1150)',
  '장애인새벽자유수영(워킹,프리)',
  '장애인아침자유수영(워킹,프리)',
  '재활건강수영A',
  '재활건강수영B',
  '저녁건강수영A19~1950',
  '저녁건강수영B20~2050',
  '직장인건강수영21-2150',
  '한마음건강수영A17~1750',
  '한마음건강수영B17~1750',
  '한마음건강수영C18~1850',
  '한마음건강수영D18-1850',
  '장애인배드민턴꿈나무',
  '스포츠스태킹',
  '스포츠스테킹A',
  '어린이요가교실',
  '장애인 라인댄스'],
 '대상': ['장애인',
  '7세~19세(지적장애)',
  '성인(복지카드소지)',
  '성인(복지카드소지)',
  '성인(복지카드소지자)',
  '성인(복지카드소지자)',
  '성인(복지카드소지)',
  '성인(복지카드소지)',
  '성인(복지카드소지자)',
  '청소년(특수학급),성인(복지카드소지자)',
  '성인(복지카드소지자)',
  '성인(복지카드소지자)',
  '성인(복지카드소지자)',
  '중.고생.성인(복지카드소지)',
  '중.고생.성인(복지카드소지)',
  '중.고생.성인(복지카드소지)',
  '중.고생.성인(복지카드소지)',
  '지적,발달',
  '

In [76]:
sort_columns = list(my_columns)
sort_columns.insert(0,"구분")
result_df = pd.DataFrame(dic)
result_df = result_df[sort_columns]
result_df = result_df.dropna(axis=0)# 결측값 행 제거

result_df.to_csv('./{}.csv'.format(sitename), sep=',', na_rep='NaN',encoding="CP949", index=False)